In [12]:
 import os
os.environ["TRANSFORMERS_NO_TF"] = "1"


In [13]:
#import sklearn as sk
import transformers
#import sentence-transformers
from  datasets import Dataset 
import torch 
import pandas as pd
from transformers import Trainer, TrainingArguments

#from tensorflow import keras

In [14]:



df = pd.read_csv("/home/profniggastein/PycharmProjects/ReducedDataset/gcj2009_balanced_exp.csv")

df = df[["flines", "solution"]].dropna()
df = df.rename(columns={"flines": "code", "solution": "label"})
df["label"] = df["label"].astype(int)


# df = df[["code", "Correct/incorrect"]].dropna()
# df = df.rename(columns={"Correct/incorrect": "label"})
# df["label"] = df["label"].astype(int)


dataset = Dataset.from_pandas(df)


In [15]:
from transformers import RobertaTokenizer
from datasets import Value

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

def preprocess(examples):
    return tokenizer(examples["code"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(preprocess, batched=True)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 3000/3000 [00:05<00:00, 501.67 examples/s]


In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
training_args = TrainingArguments(
    output_dir="./codebert-correctness-gcj(Bin-exp2)",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate= 2e-5,
    num_train_epochs=10,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

In [18]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.argmax(torch.tensor(logits),dim=1).numpy()
    preds = (probs > 0.5).astype(int)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

In [19]:




trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.703400
20,0.728800
30,0.714900
40,0.691400
50,0.719800
60,0.698900
70,0.698300
80,0.690900
90,0.694300
100,0.690500


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator i

TrainOutput(global_step=3000, training_loss=0.49345898326237997, metrics={'train_runtime': 20003.6778, 'train_samples_per_second': 1.2, 'train_steps_per_second': 0.15, 'total_flos': 3157332664320000.0, 'train_loss': 0.49345898326237997, 'epoch': 10.0})

In [20]:
    # Save the model
model.save_pretrained("./codebert-correctness-gcj(Bin-exp2)")
tokenizer.save_pretrained("./codebert-correctness-gcj(Bin-exp2)")


('./codebert-correctness-gcj(Bin-exp2)/tokenizer_config.json',
 './codebert-correctness-gcj(Bin-exp2)/special_tokens_map.json',
 './codebert-correctness-gcj(Bin-exp2)/vocab.json',
 './codebert-correctness-gcj(Bin-exp2)/merges.txt',
 './codebert-correctness-gcj(Bin-exp2)/added_tokens.json')

In [21]:
     trainer.evaluate()

/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 1.2947313785552979,
 'eval_accuracy': 0.54,
 'eval_f1': 0.5460526315789473,
 'eval_runtime': 88.5169,
 'eval_samples_per_second': 6.778,
 'eval_steps_per_second': 0.847,
 'epoch': 10.0}